In [1]:
import numpy as np
import tensorflow as tf

In [2]:
optimizer = tf.keras.optimizers.Adam()

In [3]:
def loss(inputs, outputs):
    return tf.reduce_mean(tf.square(inputs - outputs))

In [4]:
def train_step(model, inputs, outputs):
    with tf.GradientTape() as t:
        current_loss = loss(model(inputs), outputs)
        
    grads = t.gradient(current_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads,model.trainable_variables))
    return current_loss

In [5]:
class SeriesPredictor(tf.keras.Model):
    def __init__(self, input_dim, seq_size, hidden_dim=10):
        super(SeriesPredictor, self).__init__()
        self.input_dim = input_dim
        self.seq_size = seq_size
        self.hidden_dim = hidden_dim
        
        self.cell = tf.keras.layers.SimpleRNNCell(self.hidden_dim)
        self.rnn = tf.keras.layers.RNN(self.cell)
        self.fc = tf.keras.layers.Dense(self.seq_size)
                
    def call(self, x): 
        x = self.rnn(x)
        x = self.fc(x)
        
        return x

In [6]:
model = SeriesPredictor(input_dim=1, seq_size=4, hidden_dim=10)

In [7]:
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=optimizer, 
                          model=model)
manager = tf.train.CheckpointManager(ckpt, '../../models/ch16-model.ckpt', max_to_keep=3)

In [8]:
def train(model, inputs, outputs):
    for i in range(1000):
        err = train_step(model, inputs, outputs)
        ckpt.step.assign_add(1)
        if i % 100 == 0:            
            save_path = manager.save()
            print("Saved checkpoint for step {}: {}".format(int(ckpt.step), save_path))
            print("loss {:1.2f}".format(err.numpy()))
    
    return save_path

In [9]:
def test(model, inputs):
    ckpt.restore(save_path).assert_consumed()
    print("Model restored.")
    outputs = model(inputs)
    print(outputs.numpy())

In [10]:
train_x = np.asarray([[[1], [2], [5], [6]],
           [[5], [7], [7], [8]],
           [[3], [4], [5], [7]]], dtype=np.float32)
train_y = np.asarray([[1, 3, 7, 11],
           [5, 12, 14, 15],
           [3, 7, 9, 12]], dtype=np.float32)

test_x = np.asarray([[[1], [2], [3], [4]],
          [[4], [5], [6], [7]]], dtype=np.float32)

print(np.shape(train_x))
print(np.shape(train_y))
print(np.shape(test_x))


(3, 4, 1)
(3, 4)
(2, 4, 1)


In [11]:
save_path = train(model, train_x, train_y)

Saved checkpoint for step 2: ../../models/ch16-model.ckpt/ckpt-1
loss 87.04
Saved checkpoint for step 102: ../../models/ch16-model.ckpt/ckpt-2
loss 68.47
Saved checkpoint for step 202: ../../models/ch16-model.ckpt/ckpt-3
loss 53.56
Saved checkpoint for step 302: ../../models/ch16-model.ckpt/ckpt-4
loss 42.57
Saved checkpoint for step 402: ../../models/ch16-model.ckpt/ckpt-5
loss 34.20
Saved checkpoint for step 502: ../../models/ch16-model.ckpt/ckpt-6
loss 27.70
Saved checkpoint for step 602: ../../models/ch16-model.ckpt/ckpt-7
loss 22.65
Saved checkpoint for step 702: ../../models/ch16-model.ckpt/ckpt-8
loss 18.73
Saved checkpoint for step 802: ../../models/ch16-model.ckpt/ckpt-9
loss 15.71
Saved checkpoint for step 902: ../../models/ch16-model.ckpt/ckpt-10
loss 13.40


In [12]:
test(model, test_x)

Model restored.
[[2.9591253 6.653071  8.215342  7.891247 ]
 [2.9956038 6.7035503 8.27158   7.931595 ]]
